## Draw plots needed for the paper

In [ ]:
# Import required libraries
import pandas as pd
from plotUtils import PlotUtils
import numpy as np
import os
from pathlib import Path

In [ ]:
datasetToName = {"ddg": "DuckDuckGo", "bing": "Bing", "google": "Google"}
modelToName = {"resnet": "ResNet", "alexnet": "AlexNet", "vgg": "VGG"}

## Plot utilities

In [ ]:
TITLE_PAD = 30
TITLE_DIM = 30
XY_LABEL_PAD = 23
XY_LABEL_DIM = 21
XY_TICKS_DIM = 18
GRID_ALPHA = 0.2
FILL_ALPHA = 0.05
BAR_WIDTH = 0.5
LEGEND_LOC = "upper left"
LEGEND_LABEL_DIM = 15
COLOR_SCHEME = ["#91C483", "#FF6464", "#FFE162", "#C67ACE"]

FIG_SIZE = (18, 8)
FIG_BG_COLOR = None # "w"
FIG_DPI = 500
FIG_BBOX = "tight"
FIG_PAD = .3
FIG_FORMATS = ["png", "pdf"]

In [ ]:
plotter = PlotUtils(
    options={
        "titlePad": TITLE_PAD,
        "titleDim": TITLE_DIM,
        "xyLabelPad": XY_LABEL_PAD,
        "xyLabelDim": XY_LABEL_DIM,
        "xyTicksDim": XY_TICKS_DIM,
        "gridAlpha": GRID_ALPHA,
        "fillAlpha": FILL_ALPHA,
        "barWidth": BAR_WIDTH,
        "legendLoc": LEGEND_LOC,
        "legendLabelDim": LEGEND_LABEL_DIM,
        "colorScheme": COLOR_SCHEME,
        "figSize": FIG_SIZE,
        "figBGColor": FIG_BG_COLOR,
        "figDPI": FIG_DPI,
        "figBBox": FIG_BBOX,
        "figPad": FIG_PAD,
        "figFormats": FIG_FORMATS,
    }
)

In [ ]:
# Get absolute path
absPath = str(Path(os.path.abspath(os.getcwd())).absolute())
SPLIT_MEASURES_AVG_PATH = os.path.join(absPath, "SplitMeasuresAVG.csv")
SPLIT_MEASURES_ALL_PATH = os.path.join(absPath, "SplitMeasuresALL.csv")

## Plots about transferabilty

In [ ]:
avgResultDF = pd.read_csv(SPLIT_MEASURES_AVG_PATH)
avgResultDF 

In [ ]:
sameSource = [0.9704, 0.9733, 0.9504]
avgSources = [0.9734, 0.9714, 0.9517]

plotter.plotResults(
      x=np.arange(3),
      yArr=[list(sameSource), list(avgSources)],
      yLabel="F1-Score",
      xLabel="Training Set source",
      legend=["Test Set coming from the same source", "Average using all the Test Sets"],
      xTicks=list(['Bing', 'DuckDuckGo', 'Google']),
      showGrid=True,
      gridAxis="y",
      style="bar",
      saveTitle="ResNetAvg",
      yLim= [0.9,1],
      title="ResNet"
)

In [ ]:
allResultDF = pd.read_csv(SPLIT_MEASURES_ALL_PATH)
allResultDF 

In [ ]:
modelOffset = 0

bingTestSet =   [allResultDF.at[modelOffset,"Bing"]  ,allResultDF.at[12 + modelOffset,"Bing"]  ,allResultDF.at[24 + modelOffset,"Bing"]]
ddgTestSet =    [allResultDF.at[modelOffset,"DDG"]   ,allResultDF.at[12 + modelOffset,"DDG"]   ,allResultDF.at[24 + modelOffset,"DDG"]]
googleTestSet = [allResultDF.at[modelOffset,"Google"],allResultDF.at[12 + modelOffset,"Google"],allResultDF.at[24 + modelOffset,"Google"]]

plotter.plotResults(
      x=np.arange(3),
      yArr=[list(bingTestSet), list(ddgTestSet), list(ddgTestSet)],
      yLabel="F1-Score",
      xLabel="Training Set Source",
      legend=["Bing Test Set", "DuckDuckGo Test Set", "Google Test Set"],
      xTicks=list(['Bing', 'DuckDuckGo', 'Google']),
      showGrid=True,
      gridAxis="y",
      style="bar",
      saveTitle="ResNetAll",
      yLim= [0.90,1],
      title="ResNet"
      
)

In [ ]:
modelOffset = 4

bingTestSet =   [allResultDF.at[modelOffset,"Bing"]  ,allResultDF.at[12 + modelOffset,"Bing"]  ,allResultDF.at[24 + modelOffset,"Bing"]]
ddgTestSet =    [allResultDF.at[modelOffset,"DDG"]   ,allResultDF.at[12 + modelOffset,"DDG"]   ,allResultDF.at[24 + modelOffset,"DDG"]]
googleTestSet = [allResultDF.at[modelOffset,"Google"],allResultDF.at[12 + modelOffset,"Google"],allResultDF.at[24 + modelOffset,"Google"]]

plotter.plotResults(
      x=np.arange(3),
      yArr=[list(bingTestSet), list(ddgTestSet), list(ddgTestSet)],
      yLabel="F1-Score",
      xLabel="Training Set Source",
      legend=["Bing Test Set", "DuckDuckGo Test Set", "Google Test Set"],
      xTicks=list(['Bing', 'DuckDuckGo', 'Google']),
      showGrid=True,
      gridAxis="y",
      style="bar",
      saveTitle="AlexNetAll",
      yLim= [0.90,1],
      title="AlexNet"
)


In [ ]:
modelOffset = 8

bingTestSet =   [allResultDF.at[modelOffset,"Bing"]  ,allResultDF.at[12 + modelOffset,"Bing"]  ,allResultDF.at[24 + modelOffset,"Bing"]]
ddgTestSet =    [allResultDF.at[modelOffset,"DDG"]   ,allResultDF.at[12 + modelOffset,"DDG"]   ,allResultDF.at[24 + modelOffset,"DDG"]]
googleTestSet = [allResultDF.at[modelOffset,"Google"],allResultDF.at[12 + modelOffset,"Google"],allResultDF.at[24 + modelOffset,"Google"]]

plotter.plotResults(
      x=np.arange(3),
      yArr=[list(bingTestSet), list(ddgTestSet), list(ddgTestSet)],
      yLabel="F1-Score",
      xLabel="Training Set Source",
      legend=["Bing Test Set", "DuckDuckGo Test Set", "Google Test Set"],
      xTicks=list(['Bing', 'DuckDuckGo', 'Google']),
      showGrid=True,
      gridAxis="y",
      style="bar",
      saveTitle="vggAll",
      yLim= [0.90,1],
      title="VGG"
)

## Evaluations

In [ ]:
modelsEvaluationsDF = pd.read_csv(
    "../results/modelsEvaluations.csv", sep=",", index_col=0)
modelsEvaluationsDF = modelsEvaluationsDF.dropna(subset=["isMath"])
modelsEvaluationsDF["asr"] = modelsEvaluationsDF.apply(
    lambda row: 1 - row["acc"], axis=1)

modelsEvaluationsDF = modelsEvaluationsDF[modelsEvaluationsDF["attack"] != "Sharpen"]

In [ ]:
modelsEvaluationsDF

### RQ1

In [ ]:
for dataset in ["google", "ddg", "bing"]:
    datasetDF = modelsEvaluationsDF[modelsEvaluationsDF["dataset"] == dataset]
    mathMeanDF = datasetDF.groupby(["isMath", "modelDataset"]).mean()

    plotter.plotResults(
        figSize=(10, 8),
        x=np.arange(3),
        yArr=mathMeanDF.groupby("isMath")["asr"].apply(list),
        yLabel="ASR",
        xLabel="Training Set Source",
        legend=["Non math attacks", "Math attacks"],
        xTicks=['Bing', 'DuckDuckGo', 'Google'],
        showGrid=True,
        gridAxis="y",
        style="bar",
        yLim=[0, 0.14],
        saveTitle="rq1_" + dataset,
        title=datasetToName[dataset]
    )

RQ1 VARIANT

In [ ]:
modelsEvaluationsDF["sameDataset"] = modelsEvaluationsDF.apply(lambda row: row["dataset"] == row["modelDataset"],axis = 1)

sameDatasetDF = modelsEvaluationsDF.groupby(["sameDataset","attack"]).mean().sort_values(["sameDataset","attack"],ascending=True)

attacksValues = sameDatasetDF.groupby(["sameDataset"])["asr"].apply(list)

attacksNames = list(sameDatasetDF.groupby(["attack"]).mean().index)

plotter.plotResults(
    figSize=(16, 8),
    x=np.arange(len(attacksNames)),
    yArr=list(reversed(list(attacksValues))),
    xLabel="Attacks",
    legend=["Same dataset","Different Dataset"],
    xTicks=attacksNames,
    xTicksRotation=90,
    showGrid=True,
    gridAxis="y",
    style="bar",
    saveTitle="RQ1_Variant",
    title="RQ1_Variant"
)

In [ ]:
modelsEvaluationsDF

### RQ2

In [ ]:
for model in ["resnet", "alexnet", "vgg"]:
    modelDF = modelsEvaluationsDF[modelsEvaluationsDF["model"] == model]
    modelGroupedDF = modelDF.groupby(["balancing"])["acc"].apply(list)

    plotter.plotResults(
        figSize=(10, 8),
        x=np.arange(4),
        yArr=list(modelGroupedDF),
        yLabel="F1-Score",
        xLabel="Balancing",
        xTicks=list(modelGroupedDF.index),
        showGrid=True,
        gridAxis="y",
        style="boxplot",
        saveTitle="rq2_" + model,
        title=modelToName[model]
    )

### RQ3

In [ ]:
for isMath in [True]:
    
    datasetDF = modelsEvaluationsDF[modelsEvaluationsDF["isMath"] == isMath]

    alexnetDF = datasetDF[datasetDF["model"] == "alexnet"]
    alexnetMeanDF = alexnetDF.groupby(["advModel"]).mean()

    resnetDF = datasetDF[datasetDF["model"] == "resnet"]
    resnetMeanDF = resnetDF.groupby(["advModel"]).mean()

    vggDF = datasetDF[datasetDF["model"] == "vgg"]
    vggMeanDF =vggDF.groupby(["advModel"]).mean()

    if isMath == True:
        graph_title = "Mathematical Attacks"
        file_title = "math"
    else:
        graph_title = "Non Mathematical Attacks"
        file_title = "nonmath"

    # alexnetDF = datasetDF[datasetDF["model"] == True]
    # mathMeanDF = mathDF.groupby(["advModel"]).mean()

    # nonMathDF = datasetDF[datasetDF["isMath"] == False]
    # nonMathMeanDF = nonMathDF.groupby(["modelDataset"]).mean()

    plotter.plotResults(
        figSize=(10, 8),
        x=np.arange(3),
        yArr=[list(1 - alexnetMeanDF["acc"]), list(1 - resnetMeanDF["acc"]), list(1 - vggMeanDF["acc"])],
        yLabel="ASR",
        xLabel="Attack Model Source",
        legend=["AlexNet", "ResNet", "VGG"],
        xTicks=["AlexNet", "ResNet", "VGG"],
        showGrid=True,
        gridAxis="y",
        style="bar",
        yLim=[0, 0.2],
        saveTitle="rq3_" + file_title,
        title=graph_title
    )

In [ ]:
datasetDF = modelsEvaluationsDF[modelsEvaluationsDF["isMath"] == False]

alexnetDF = datasetDF[datasetDF["model"] == "alexnet"]
alexnetMeanDF = alexnetDF.groupby(["model"]).mean()

resnetDF = datasetDF[datasetDF["model"] == "resnet"]
resnetMeanDF = resnetDF.groupby(["model"]).mean()

vggDF = datasetDF[datasetDF["model"] == "vgg"]
vggMeanDF =vggDF.groupby(["model"]).mean()

graph_title = "Non Mathematical Attacks"
file_title = "nonmath"

# alexnetDF = datasetDF[datasetDF["model"] == True]
# mathMeanDF = mathDF.groupby(["advModel"]).mean()

# nonMathDF = datasetDF[datasetDF["isMath"] == False]
# nonMathMeanDF = nonMathDF.groupby(["modelDataset"]).mean()

plotter.plotResults(
    figSize=(10, 8),
    x=np.arange(3),
    yArr=[list(1 - alexnetMeanDF["acc"]), list(1 - resnetMeanDF["acc"]), list(1 - vggMeanDF["acc"])],
    yLabel="ASR",
    xLabel="Attack Model Source",
    legend=["AlexNet", "ResNet", "VGG"],
    xTicks=["AlexNet", "ResNet", "VGG"],
    showGrid=True,
    gridAxis="y",
    style="bar",
    yLim=[0, 0.2],
    saveTitle="rq3_" + file_title,
    title=graph_title
)

### RQ3 Variant

In [ ]:
modelsEvaluationsDF["sameModel"] = modelsEvaluationsDF.apply(lambda row: row["advModel"] == row["model"],axis = 1)

mathDF = modelsEvaluationsDF[modelsEvaluationsDF["isMath"] == True]

sameModelDF = mathDF.groupby(["sameModel","attack"]).mean().sort_values(["sameModel","attack"],ascending=True)

attacksValues = sameModelDF.groupby(["sameModel"])["asr"].apply(list)

attacksNames = list(sameModelDF.groupby(["attack"]).mean().index)

plotter.plotResults(
    figSize=(16, 8),
    x=np.arange(len(attacksNames)),
    yArr=list(reversed(list(attacksValues))),
    xLabel="Attacks",
    legend=["Same Model","Different Model"],
    xTicks=attacksNames,
    xTicksRotation=90,
    showGrid=True,
    gridAxis="y",
    style="bar",
    saveTitle="RQ3_Variant",
    title="RQ3_Variant"
)

### RQ4

In [ ]:
modelsEvaluationsDF["sameDataset"] = modelsEvaluationsDF.apply(lambda row: row["dataset"] == row["modelDataset"],axis = 1)

sameDatasetDF = modelsEvaluationsDF.groupby(["sameDataset","model"]).mean().sort_values(["sameDataset","model"],ascending=True)

modelsValues = sameDatasetDF.groupby(["sameDataset"])["asr"].apply(list)

modelsNames = list(sameDatasetDF.groupby(["model"]).mean().index)

plotter.plotResults(
    figSize=(12, 8),
    x=np.arange(len(modelsNames)),
    yArr=list(reversed(list(modelsValues))),
    xLabel="Models",
    legend=["Same dataset","Different Dataset"],
    xTicks=[modelToName[name] for name in modelsNames],
    showGrid=True,
    gridAxis="y",
    style="bar",
    yLim=[0,0.14],
    saveTitle="RQ4",
    title="RQ4"
)